## Example 3: Multi-Product Production and Distribution Problem

This example demonstrates a complex optimization problem involving:
- **Multiple production facilities** (A and B)
- **Multiple production lines** at each facility
- **Multiple products** (represented by index combinations)
- **Transportation** between facilities
- **Capacity constraints** with efficiency factors
- **Production-distribution linkages**

### Problem Description

**Decision Variables:**
- `x[i,j,k,m]`: Production quantity from facility i, using configuration j-k, for product type m
  - i ∈ {A, B} (facilities)
  - j ∈ {1, 2, 3, 4, 5} (source configuration)
  - k ∈ {1, 2, 3, 4, 5} (destination configuration)
  - m ∈ {1, 2, 3} (product type)
  
- `y[i1,i2]`: Transportation flow from facility i1 to facility i2
  - Specifically: y[A,B] and y[B,A]

**Objective Function:**
Maximize total profit = Revenue - Transportation Costs - Production Costs

- **Revenue Terms:**
  - 1550 × Σ x[i,j,1,1] (Product type 1, config 1→1)
  - 1600 × Σ x[i,j,1,2] (Product type 2, config 1→2)
  - 1400 × Σ x[i,j,2,3] (Product type 3, config 2→3)
  - 1000 × Σ x[i,j,3,4] (Product type 4, config 3→4)
  - 850 × Σ x[i,j,3,5] (Product type 5, config 3→5)

- **Transportation Costs:**
  - 2 × (y[A,B] + y[B,A])

- **Production Costs:**
  - Facility A, Line 1: 250 × Σ x[A,1,k,m]
  - Facility A, Line 2: 320 × Σ x[A,2,k,m]
  - Facility B, Line 1: 260 × Σ x[B,1,k,m]
  - Facility B, Line 2: 210 × Σ x[B,2,k,m]
  - Facility B, Line 3: 350 × Σ x[B,3,k,m]

### Step 1: Import Required Libraries

In [ ]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import pandas as pd

### Step 2: Define the Model and Sets

We create a concrete model and define all the index sets for our problem.

In [ ]:
# Create a concrete model
model = pyo.ConcreteModel(name="Production_Distribution_Problem")

# Define Sets
model.I = pyo.Set(initialize=['A', 'B'], doc='Facilities')
model.J = pyo.Set(initialize=[1, 2, 3, 4, 5], doc='Source configurations')
model.K = pyo.Set(initialize=[1, 2, 3, 4, 5], doc='Destination configurations')
model.M = pyo.Set(initialize=[1, 2, 3], doc='Product types')

print("Sets defined successfully!")
print(f"Facilities (I): {list(model.I)}")
print(f"Source configs (J): {list(model.J)}")
print(f"Destination configs (K): {list(model.K)}")
print(f"Product types (M): {list(model.M)}")

### Step 3: Define Decision Variables

We have two types of decision variables:
1. **x[i,j,k,m]**: Production quantities (4-dimensional variable)
2. **y[i1,i2]**: Transportation flows between facilities

In [ ]:
# Define Variables
model.x = pyo.Var(model.I, model.J, model.K, model.M, 
                  within=pyo.NonNegativeReals, 
                  doc='Production quantity from facility i, config j->k, product type m')

model.y_AB = pyo.Var(within=pyo.NonNegativeReals, 
                     doc='Transportation from A to B')

model.y_BA = pyo.Var(within=pyo.NonNegativeReals, 
                     doc='Transportation from B to A')

print("Decision variables defined successfully!")
print(f"x has {len(model.I) * len(model.J) * len(model.K) * len(model.M)} potential variables")
print(f"y has 2 variables (y_AB and y_BA)")

### Step 4: Define the Objective Function

**Maximize:** Revenue - Transportation Costs - Production Costs

The objective is broken down into three components for clarity.

In [ ]:
def objective_rule(model):
    # Revenue terms (different product configurations have different prices)
    revenue = (1550 * sum(model.x[i, j, 1, 1] for i in model.I for j in model.J) +
               1600 * sum(model.x[i, j, 1, 2] for i in model.I for j in model.J) +
               1400 * sum(model.x[i, j, 2, 3] for i in model.I for j in model.J) +
               1000 * sum(model.x[i, j, 3, 4] for i in model.I for j in model.J) +
               850 * sum(model.x[i, j, 3, 5] for i in model.I for j in model.J))
    
    # Transportation costs (cost per unit transported between facilities)
    transport_cost = 2 * (model.y_AB + model.y_BA)
    
    # Production costs (different production lines have different costs)
    production_cost = (250 * sum(model.x['A', 1, k, m] for k in model.K for m in model.M) +
                       320 * sum(model.x['A', 2, k, m] for k in model.K for m in model.M) +
                       260 * sum(model.x['B', 1, k, m] for k in model.K for m in model.M) +
                       210 * sum(model.x['B', 2, k, m] for k in model.K for m in model.M) +
                       350 * sum(model.x['B', 3, k, m] for k in model.K for m in model.M))
    
    return revenue - transport_cost - production_cost

model.objective = pyo.Objective(rule=objective_rule, sense=pyo.maximize, doc='Maximize profit')
print("Objective function defined successfully!")

### Step 5: Define Capacity Constraints

Each production line has a capacity limit. The efficiency factors (0.65, 0.80, 0.76, 0.78) represent the yield rates - we need to account for waste/inefficiency in production.

In [ ]:
# Individual capacity constraints for each production line
def capacity_A1_rule(model):
    """Facility A, Line 1 capacity (accounting for 0.65 efficiency)"""
    return sum(model.x['A', 1, k, m] for k in model.K for m in model.M) / 0.65 <= 300

def capacity_A2_rule(model):
    """Facility A, Line 2 capacity (accounting for 0.80 efficiency)"""
    return sum(model.x['A', 2, k, m] for k in model.K for m in model.M) / 0.80 <= 280

def capacity_B1_rule(model):
    """Facility B, Line 1 capacity (accounting for 0.76 efficiency)"""
    return sum(model.x['B', 1, k, m] for k in model.K for m in model.M) / 0.76 <= 400

def capacity_B2_rule(model):
    """Facility B, Line 2 capacity (accounting for 0.76 efficiency)"""
    return sum(model.x['B', 2, k, m] for k in model.K for m in model.M) / 0.76 <= 250

def capacity_B3_rule(model):
    """Facility B, Line 3 capacity (accounting for 0.78 efficiency)"""
    return sum(model.x['B', 3, k, m] for k in model.K for m in model.M) / 0.78 <= 500

# Register constraints
model.capacity_A1 = pyo.Constraint(rule=capacity_A1_rule)
model.capacity_A2 = pyo.Constraint(rule=capacity_A2_rule)
model.capacity_B1 = pyo.Constraint(rule=capacity_B1_rule)
model.capacity_B2 = pyo.Constraint(rule=capacity_B2_rule)
model.capacity_B3 = pyo.Constraint(rule=capacity_B3_rule)

print("Individual capacity constraints defined successfully!")

### Step 6: Total Capacity Constraint

The total effective capacity across all production lines cannot exceed 850 units.

In [ ]:
def total_capacity_rule(model):
    """Total capacity constraint across all production lines"""
    total = (sum(model.x['A', 1, k, m] for k in model.K for m in model.M) / 0.65 +
             sum(model.x['A', 2, k, m] for k in model.K for m in model.M) / 0.80 +
             sum(model.x['B', 1, k, m] for k in model.K for m in model.M) / 0.76 +
             sum(model.x['B', 2, k, m] for k in model.K for m in model.M) / 0.76 +
             sum(model.x['B', 3, k, m] for k in model.K for m in model.M) / 0.78)
    return total <= 850

model.total_capacity = pyo.Constraint(rule=total_capacity_rule)
print("Total capacity constraint defined successfully!")

### Step 7: Transportation Balance Constraint

This constraint links production at Facility A (lines 1 and 2) with the transportation flows. The factor 0.9 might represent a transportation efficiency or loss factor.

In [ ]:
def transport_balance_rule(model):
    """Transportation balance constraint linking production and flows"""
    production_A = (50 * sum(model.x['A', 1, k, m] for k in model.K for m in model.M) +
                    62 * sum(model.x['A', 2, k, m] for k in model.K for m in model.M))
    
    transport_balance = 0.9 * model.y_BA - model.y_AB
    
    return production_A + transport_balance <= 1300

model.transport_balance = pyo.Constraint(rule=transport_balance_rule)
print("Transportation balance constraint defined successfully!")

### Step 8: Demand/Production Limits

These constraints limit the production of specific product-configuration combinations, possibly representing market demand limits or other business constraints.

In [ ]:
def demand_11_rule(model):
    """Maximum production for config 1->1"""
    return sum(model.x[i, j, 1, 1] for i in model.I for j in model.J) <= 150

def demand_12_rule(model):
    """Maximum production for config 1->2"""
    return sum(model.x[i, j, 1, 2] for i in model.I for j in model.J) <= 100

def demand_21_rule(model):
    """Maximum production for config 2->1"""
    return sum(model.x[i, j, 2, 1] for i in model.I for j in model.J) <= 200

def demand_31_rule(model):
    """Maximum production for config 3->1"""
    return sum(model.x[i, j, 3, 1] for i in model.I for j in model.J) <= 50

def demand_32_rule(model):
    """Maximum production for config 3->2"""
    return sum(model.x[i, j, 3, 2] for i in model.I for j in model.J) <= 400

# Register constraints
model.demand_11 = pyo.Constraint(rule=demand_11_rule)
model.demand_12 = pyo.Constraint(rule=demand_12_rule)
model.demand_21 = pyo.Constraint(rule=demand_21_rule)
model.demand_31 = pyo.Constraint(rule=demand_31_rule)
model.demand_32 = pyo.Constraint(rule=demand_32_rule)

print("Demand/production limit constraints defined successfully!")

### Step 9: Product Mix Constraints

These constraints enforce specific ratios in production. For example, 20% of all production from Facility A, Line 1 must be of product type 1, 30% must be product type 2, and 50% must be product type 3.

**Note:** The original formulation appears to have a typo (A3km doesn't exist since j only goes to 2 for facility A based on cost structure). I'll implement what makes sense based on the pattern.

In [ ]:
def product_mix_A1_type1_rule(model):
    """20% of Facility A, Line 1 production must be product type 1"""
    total_A1 = sum(model.x['A', 1, k, m] for k in model.K for m in model.M)
    type1_A1 = sum(model.x['A', 1, k, 1] for k in model.K)
    return type1_A1 == 0.2 * total_A1

def product_mix_A1_type2_rule(model):
    """30% of Facility A, Line 2 production must be product type 2"""
    total_A2 = sum(model.x['A', 2, k, m] for k in model.K for m in model.M)
    type2_A2 = sum(model.x['A', 1, k, 2] for k in model.K)
    return type2_A2 == 0.3 * total_A2

def product_mix_A1_type3_rule(model):
    """50% of remaining Facility A production must be product type 3"""
    # Assuming this applies to combined A1 and A2 production
    total_A = sum(model.x['A', j, k, m] for j in [1, 2] for k in model.K for m in model.M)
    type3_A = sum(model.x['A', 1, k, 3] for k in model.K)
    return type3_A == 0.5 * total_A

# Register constraints
model.product_mix_A1_1 = pyo.Constraint(rule=product_mix_A1_type1_rule)
model.product_mix_A1_2 = pyo.Constraint(rule=product_mix_A1_type2_rule)
model.product_mix_A1_3 = pyo.Constraint(rule=product_mix_A1_type3_rule)

print("Product mix constraints defined successfully!")
print("\nNote: The product mix constraints enforce production ratios:")
print("  - 20% of A1 production → product type 1")
print("  - 30% of A2 production → product type 2")
print("  - 50% of total A production → product type 3")

### Step 10: Solve the Model

Now we'll solve the optimization problem using a solver. We'll use GLPK (open-source) or CBC as they're commonly available.

In [ ]:
# Display model summary before solving
print("="*60)
print("MODEL SUMMARY")
print("="*60)
print(f"Number of variables: {len([v for v in model.component_objects(pyo.Var, active=True)])}")
print(f"Number of constraints: {len([c for c in model.component_objects(pyo.Constraint, active=True)])}")
print(f"Objective sense: Maximize")
print("="*60)

# Solve the model
solver = SolverFactory('glpk')  # You can also try 'cbc', 'gurobi', 'cplex', etc.

print("\nSolving the optimization problem...")
results = solver.solve(model, tee=True)  # tee=True shows solver output

# Check solver status
print("\n" + "="*60)
print("SOLVER STATUS")
print("="*60)
print(f"Solver Status: {results.solver.status}")
print(f"Termination Condition: {results.solver.termination_condition}")
print("="*60)

### Step 11: Display Results

Let's examine the optimal solution in a structured way.

In [ ]:
# Display optimal objective value
print("\n" + "="*60)
print("OPTIMAL SOLUTION")
print("="*60)
print(f"Maximum Profit: ${pyo.value(model.objective):,.2f}")
print("="*60)

# Display transportation variables
print("\n" + "="*60)
print("TRANSPORTATION FLOWS")
print("="*60)
print(f"From A to B (y_AB): {pyo.value(model.y_AB):,.2f} units")
print(f"From B to A (y_BA): {pyo.value(model.y_BA):,.2f} units")
print("="*60)

# Collect non-zero production variables
production_data = []
for i in model.I:
    for j in model.J:
        for k in model.K:
            for m in model.M:
                value = pyo.value(model.x[i, j, k, m])
                if value > 0.01:  # Only show non-zero values
                    production_data.append({
                        'Facility': i,
                        'Source_Config': j,
                        'Dest_Config': k,
                        'Product_Type': m,
                        'Quantity': round(value, 2)
                    })

if production_data:
    df_production = pd.DataFrame(production_data)
    print("\n" + "="*60)
    print("NON-ZERO PRODUCTION QUANTITIES")
    print("="*60)
    print(df_production.to_string(index=False))
    print("="*60)
else:
    print("\nNo production (all variables are zero)")

### Step 12: Analyze Production by Facility and Line

In [ ]:
# Summary by production line
print("\n" + "="*60)
print("PRODUCTION SUMMARY BY LINE")
print("="*60)

lines = [
    ('A', 1, 0.65),
    ('A', 2, 0.80),
    ('B', 1, 0.76),
    ('B', 2, 0.76),
    ('B', 3, 0.78)
]

for facility, line, efficiency in lines:
    total = sum(pyo.value(model.x[facility, line, k, m]) 
                for k in model.K for m in model.M)
    capacity_used = total / efficiency
    print(f"Facility {facility}, Line {line}:")
    print(f"  Production: {total:,.2f} units")
    print(f"  Capacity Used: {capacity_used:,.2f} / {300 if (facility=='A' and line==1) else 280 if (facility=='A' and line==2) else 400 if (facility=='B' and line==1) else 250 if (facility=='B' and line==2) else 500} units")
    print(f"  Efficiency: {efficiency*100}%")
    print()

print("="*60)

### Step 13: Verify Constraints

Let's verify that key constraints are satisfied.

In [ ]:
print("\n" + "="*60)
print("CONSTRAINT VERIFICATION")
print("="*60)

# Total capacity check
total_cap = (sum(pyo.value(model.x['A', 1, k, m]) for k in model.K for m in model.M) / 0.65 +
             sum(pyo.value(model.x['A', 2, k, m]) for k in model.K for m in model.M) / 0.80 +
             sum(pyo.value(model.x['B', 1, k, m]) for k in model.K for m in model.M) / 0.76 +
             sum(pyo.value(model.x['B', 2, k, m]) for k in model.K for m in model.M) / 0.76 +
             sum(pyo.value(model.x['B', 3, k, m]) for k in model.K for m in model.M) / 0.78)

print(f"Total Capacity Used: {total_cap:.2f} / 850.00 units")

# Check demand constraints
demands = {
    '1→1': (sum(pyo.value(model.x[i, j, 1, 1]) for i in model.I for j in model.J), 150),
    '1→2': (sum(pyo.value(model.x[i, j, 1, 2]) for i in model.I for j in model.J), 100),
    '2→1': (sum(pyo.value(model.x[i, j, 2, 1]) for i in model.I for j in model.J), 200),
    '3→1': (sum(pyo.value(model.x[i, j, 3, 1]) for i in model.I for j in model.J), 50),
    '3→2': (sum(pyo.value(model.x[i, j, 3, 2]) for i in model.I for j in model.J), 400),
}

print("\nDemand Constraints:")
for config, (actual, limit) in demands.items():
    status = "✓" if actual <= limit + 0.01 else "✗"
    print(f"  Config {config}: {actual:.2f} / {limit:.2f} {status}")

print("="*60)

### Step 14: Cost-Benefit Breakdown

Let's break down where the profit comes from.

In [ ]:
# Calculate components
revenue_components = {
    '1→1 ($1550)': 1550 * sum(pyo.value(model.x[i, j, 1, 1]) for i in model.I for j in model.J),
    '1→2 ($1600)': 1600 * sum(pyo.value(model.x[i, j, 1, 2]) for i in model.I for j in model.J),
    '2→3 ($1400)': 1400 * sum(pyo.value(model.x[i, j, 2, 3]) for i in model.I for j in model.J),
    '3→4 ($1000)': 1000 * sum(pyo.value(model.x[i, j, 3, 4]) for i in model.I for j in model.J),
    '3→5 ($850)': 850 * sum(pyo.value(model.x[i, j, 3, 5]) for i in model.I for j in model.J),
}

production_costs = {
    'A-Line1 ($250)': 250 * sum(pyo.value(model.x['A', 1, k, m]) for k in model.K for m in model.M),
    'A-Line2 ($320)': 320 * sum(pyo.value(model.x['A', 2, k, m]) for k in model.K for m in model.M),
    'B-Line1 ($260)': 260 * sum(pyo.value(model.x['B', 1, k, m]) for k in model.K for m in model.M),
    'B-Line2 ($210)': 210 * sum(pyo.value(model.x['B', 2, k, m]) for k in model.K for m in model.M),
    'B-Line3 ($350)': 350 * sum(pyo.value(model.x['B', 3, k, m]) for k in model.K for m in model.M),
}

transport_cost = 2 * (pyo.value(model.y_AB) + pyo.value(model.y_BA))

print("\n" + "="*60)
print("PROFIT BREAKDOWN")
print("="*60)

total_revenue = sum(revenue_components.values())
print(f"\nTOTAL REVENUE: ${total_revenue:,.2f}")
for product, revenue in revenue_components.items():
    if revenue > 0:
        print(f"  {product}: ${revenue:,.2f}")

total_prod_cost = sum(production_costs.values())
print(f"\nTOTAL PRODUCTION COSTS: ${total_prod_cost:,.2f}")
for line, cost in production_costs.items():
    if cost > 0:
        print(f"  {line}: ${cost:,.2f}")

print(f"\nTRANSPORTATION COSTS: ${transport_cost:,.2f}")

print(f"\n{'='*60}")
print(f"NET PROFIT: ${total_revenue - total_prod_cost - transport_cost:,.2f}")
print("="*60)

### Key Takeaways

**This optimization problem demonstrates:**

1. **Multi-dimensional decision variables** - The 4D variable `x[i,j,k,m]` represents complex production scenarios

2. **Capacity constraints with efficiency factors** - Real-world production isn't 100% efficient

3. **Multiple constraint types:**
   - Individual line capacities
   - Aggregate capacity limits
   - Demand/market limits
   - Product mix requirements
   - Transportation balance

4. **Complex objective function** - Balancing revenue from multiple products against various cost types

5. **Trade-offs** - The solver finds the optimal balance between:
   - High-revenue products vs. production costs
   - Transportation costs vs. production distribution
   - Capacity utilization vs. efficiency losses

**Extensions you could try:**
- Add inventory costs
- Include setup costs for switching between products
- Add time periods (multi-period planning)
- Include uncertainty in demand or prices
- Add quality constraints
- Model economies of scale